In [43]:
import matplotlib.pyplot as plt
from scipy import optimize
import numpy as np

%matplotlib qt

In [2]:
def weight_i(Rp_pass,alpha_i):
    '''
    Rp: total illumination of planet in counts
    alpha_i: total sky counts in observation i
    '''
    
    return np.power(Rp_pass+alpha_i,-0.5)

In [3]:
def sewm(weights_array_pass):
    '''
    sewm = "standard error of the weighted mean"
    weights_array: array of weights
    '''
    
    return np.divide(1.,np.sqrt(np.nansum(weights_array_pass)))

In [4]:
np.random.rand()

0.7252648712892664

In [65]:
# Over N nights, how does the sewm change for the same level of contrast?

N = 30 # number of nights
sewm_constant_array = np.nan*np.ones(N) # initialize, for N nights
sewm_random_array = np.nan*np.ones(N) # initialize
weight_constant_array = np.nan*np.ones(N) # initialize
weight_random_array = np.nan*np.ones(N) # initialize
inv_sqrt_array = np.nan*np.ones(N) # initialize
Rp = 1 # planet brightness

# loop over each night, re-calculating the sewm for the cumulative night 
for night_num in range(0,len(sewm_constant_array)):
    
    alpha_i = 5 # sky level that night (if constant)
    Rp_i = Rp # planet that night

    # for constant sky
    weight_constant_array[night_num] = weight_i(Rp_i,alpha_i)
    sewm_constant_array[night_num] = sewm(weight_constant_array)
    
    # for varying sky
    weight_random_array[night_num] = weight_i(Rp_i,alpha_i+5*(np.random.rand()-0.5))
    sewm_random_array[night_num] = sewm(weight_random_array)
    
    # 1/sqrt(N) curve
    inv_sqrt_array[night_num] = np.divide(1.,np.sqrt(night_num))
    
    # scale so as to fit the other two curves
    inv_sqrt_array_constant = np.multiply(inv_sqrt_array,np.divide(sewm_constant_array[1],inv_sqrt_array[1]))
    inv_sqrt_array_random = np.multiply(inv_sqrt_array,np.divide(sewm_random_array[1],inv_sqrt_array[1]))

/Users/nyumbani/anaconda3/envs/py366/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide


In [66]:
np.random.rand(10)-0.5

array([-0.29689813,  0.11150785,  0.35512165,  0.48658687, -0.39928892,
        0.40189723, -0.09032709, -0.49007478,  0.45262821, -0.34296679])

In [67]:
# best fits to square roots

def sqrt_func(x, a, b):
    return a * np.divide(1.,np.sqrt(x)) + b

x_data_1 = np.arange(len(sewm_constant_array))[1:]
y_data_1 = sewm_constant_array[1:]
params_1, params_covariance_1 = optimize.curve_fit(sqrt_func, x_data_1, y_data_1, p0=None)

x_data_2 = np.arange(len(sewm_random_array))[1:]
y_data_2 = sewm_random_array[1:]
params_2, params_covariance_2 = optimize.curve_fit(sqrt_func, x_data_2, y_data_2, p0=None)

In [68]:
# plot 

plt.plot(np.arange(len(sewm_constant_array))[1:],sewm_constant_array[1:],
         marker="o",label="std error of the weighted mean (same sky)")
plt.plot(np.arange(len(sewm_random_array))[1:],sewm_random_array[1:],
         marker="o",label="std error of the weighted mean (random sky)")

plt.plot(x_data_1, sqrt_func(x_data_1,params_1[0],params_1[1]),label="1/sqrt(N) (scaled to same sky)")
plt.plot(x_data_2, sqrt_func(x_data_2,params_2[0],params_2[1]),label="1/sqrt(N) (scaled to random sky)")

plt.yscale('log')
plt.xlabel("Number nights")
plt.legend()
plt.show()